In [4]:
%cd ../../../

/data/group_data/l3lab/pranjala/deepscaler


/home/pranjala/miniconda3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
from datasets import load_dataset
import random

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Idavidrein/gpqa", "gpqa_diamond")

In [22]:
ds['train'][0]['Question']

'Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could be their energy difference so that they can be clearly resolved?\n'

In [2]:
import pandas as pd

In [4]:
#   for num_tokens in [512, 1024, 2048, 3600, 4096, 8192, 16384, 32768]:
for num_tokens in [128, 256]:

    # for num_tokens in [32768]:
    # for num_tokens in [-1]:
    all_data = []
    for i in range(len(ds['train'])):
        correct_answer = ds['train'][i]['Correct Answer'].strip()
        incorrect_answers = []
        incorrect_answers.append(ds['train'][i]['Incorrect Answer 1'].strip())
        incorrect_answers.append(ds['train'][i]['Incorrect Answer 2'].strip())
        incorrect_answers.append(ds['train'][i]['Incorrect Answer 3'].strip())
        # Get shuffled choices 
        shuffled_choices = incorrect_answers + [correct_answer]
        random.shuffle(shuffled_choices)

        # Add Options to the question
        question = ds['train'][i]['Question']
        question += f"\n\nOptions:\n"
        for i, choice in enumerate(shuffled_choices):
            question += f"{chr(65 + i)}. {choice}\n"
        correct_choice = chr(65 + shuffled_choices.index(correct_answer))
        question = f"{question}"+"\n\nLet's think step by step and output the final answer (eg, A, B, C, D) within \\boxed{}." + f" Think for {num_tokens} tokens."


        all_data.append({
                    "data_source": "gpqa",
                    "prompt": [{
                        "role": "user",
                        "content": question
                    }],
                    "ability": "math",
                    "reward_model": {
                        "style": "rule",
                        "ground_truth": correct_choice,
                        "num_tokens": num_tokens
                    },
                    "extra_info": {
                        'split': 'test',
                        'index': i
                    }
                })
    # import pandas as pd
    pd.DataFrame(all_data).to_parquet(f'~/deepscaler/data_{num_tokens}/gpqa.parquet')
    # pd.DataFrame(all_data).to_parquet(f'~/deepscaler/data_{num_tokens}/lsat.parquet')
    

In [44]:
ds_lsat['test'][i]['gold'][0]

0

In [31]:
pd.read_parquet(f'~/deepscaler/data_4096/lsat.parquet')

,data_source,prompt,ability,reward_model,extra_info
0,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
1,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
2,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
3,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
4,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
...,...,...,...,...,...
193,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
194,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
195,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"
196,gpqa,"[{'content': 'Of the eight students—George, He...",math,"{'ground_truth': 'A', 'num_tokens': 4096, 'sty...","{'index': 3, 'split': 'test'}"


In [27]:
pd.DataFrame(all_data).to_parquet(f'~/deepscaler/data/lsat.parquet')

## LSAT

In [34]:
from datasets import load_dataset

ds = load_dataset("dmayhem93/agieval-lsat-ar")

In [35]:
ds_lsat = load_dataset("dmayhem93/agieval-lsat-ar")

In [17]:
import pandas as pd

all_data = []
for i, row in enumerate(ds['test']):
    # all_data.append({
    #     "data_source": "gpqa",
    #     "prompt": [{
    #         "role": "user",
    #         # "content": row['query'] + "\n\nLet's think step by step and output the final answer (eg, A, B, C, D) within \\boxed{}."
    #         "content": ds_gpqa['train'][0]['Question']
    #     }],
    #     "ability": "math",
    #     "reward_model": {
    #         "style": "rule",
    #         # "ground_truth": chr(65 + row['gold'][0]),
    #         # "ground_truth": chr(65 + row['gold'][0]),

    #         "num_tokens": 2048
    #     },
    #     "extra_info": {
    #         'split': 'test',
    #         'index': i
    #     }
    # })
    all_data.append({
        "data_source": "gpqa",
        "prompt": [{
            "role": "user",
            "content": ds_gpqa['train'][0]['Question']
        }],
        "ability": "math",
        "reward_model": {
            "style": "rule",
            "ground_truth": 'A',
            "num_tokens": 32768
        },
        "extra_info": {
            'split': 'test',
            'index': i
        }
    })

pd.DataFrame(all_data).to_parquet(f'~/deepscaler/data/lsat.parquet')

In [6]:
!cp ~/deepscaler/data/gpqa.parquet ~/deepscaler/data/lsat.parquet

## MMLU

In [1]:
from datasets import load_dataset
import random
ds = load_dataset("cais/mmlu", "all")

In [2]:
ds['test'][-1]

{'question': ' The numen of Augustus referred to which of the following characteristics?',
 'subject': 'world_religions',
 'choices': ['Divine power',
  'Sexual virility',
  'Military acumen',
  'Philosophical intellect'],
 'answer': 0}

In [33]:
ds['test'][-1]

{'question': ' The numen of Augustus referred to which of the following characteristics?',
 'subject': 'world_religions',
 'choices': ['Divine power',
  'Sexual virility',
  'Military acumen',
  'Philosophical intellect'],
 'answer': 0}

In [10]:
ds_gpqa = load_dataset("Idavidrein/gpqa", "gpqa_diamond")

In [13]:
all_data = []
for i, row in enumerate(ds['test']):
    options = row['choices']
    options_str  = ""
    for j in range(len(options)):
        options_str += f"{chr(65 + j)}. {options[j]}\n"
    all_data.append({
        "data_source": "mmlu",
        "prompt": [{
            "role": "user",
            "content": row['question'] + "\n\nOptions:\n" + options_str
        }],
        "ability": "mmlu",
        "reward_model": {
            "style": "rule",
            "ground_truth": chr(65 + row['answer']),
            "num_tokens": -1
        },
        "extra_info": {
            'split': 'test',
            'index': i,
            'subject': row['subject']
        }
    })

# Choose random 1000 rows
indices = random.sample(range(len(all_data)), 1000)
all_data_small = [all_data[i] for i in indices]
pd.DataFrame(all_data_small).to_parquet(f'~/deepscaler/data/mmlu.parquet')

KeyError: 'answer'

In [31]:
print(pd.DataFrame(all_data)['reward_model'][24])

{'style': 'rule', 'ground_truth': 'B', 'num_tokens': -1}


In [27]:
print(pd.DataFrame(all_data)['prompt'][12][0]['content'])

Find the order of the factor group (Z_11 x Z_15)/(<1, 1>)

Options:
A. 1
B. 2
C. 5
D. 11



In [20]:
incorrect_answers

['10^-11 eV', '10^-8 eV', '10^-9 eV']

In [12]:
ds['train'][0]

{'Pre-Revision Question': 'Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could be their energy difference so that they be clearly resolved?\n',
 'Pre-Revision Correct Answer': '10^-4 ev',
 'Pre-Revision Incorrect Answer 1': '10^-8 ev',
 'Pre-Revision Incorrect Answer 2': '10^-9 ev',
 'Pre-Revision Incorrect Answer 3': '10^-11 ev',
 'Pre-Revision Explanation': 'According to uncertainty principle, Delta E* Delta t=hbar/2. Delta t is the lifetime and Delta E is the width of the energy level . With Delta t=10^-9 s==> Delta E1= 3.3 *10^-7 ev. And Delta t=10^-11 s gives Delta E2=3.3*10^-8 ev.\nTherefore, the energy difference between the two states must be significantly greater than 10^-7 ev. So the answer is 10^-4 ev.',
 'Self-reported question-writing time (minutes)': 30.0,
 'Question': 'Two quantum states with energies E1 and E2 have a lifetime 

In [54]:
print(response['reward_model'][0]['ground_truth'])

A


In [19]:
import numpy as np
response['token_lengths'].apply(np.mean).mean()

8468.054292929293

In [30]:
[response['reward_model'][i]['ground_truth'] for i in range(len(response))]

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A'

In [29]:
import pandas as pd
# response = pd.read_parquet('gs_newr2_hotinit/step_700_2048/gpqa.parquet')
response = pd.read_parquet('gs_newr2_hotinit/step_700_4096/lsat.parquet')
# response = pd.read_parquet('deepseek-r1-1.5b-baseline/16384/gpqa.parquet')
print(response['responses'][0][14])

<think>
Okay, so I have this problem about eight students: George, Helen, Irving, Kyle, Lenore, Nina, Olivia, and Robert. They need to give reports on three days: Monday, Tuesday, and Wednesday. Each day has one morning and one afternoon report. So, that's six reports in total, with two reports per day.

The conditions given are:

1. Exactly six will give reports, two each day.
2. Tuesday is the only day George can give a report.
3. Neither Olivia nor Robert can give an afternoon report.
4. If Nina gives a report, then on the next day Helen and Irving must both give reports, unless Nina's report is on Wednesday.

Then, the question is: If George, Nina, and Robert give reports and they do so on different days from one another, which one of the following could be true? The options are A to E: A) Helen gives a report on Wednesday. B) Nina gives a report on Monday. C) Nina gives a report on Tuesday. D) Olivia gives a report on Monday. E) Robert gives a report on Wednesday.

We need to dete

In [11]:
def get_model_choice(res):
    for i in range(len(res)-1, -1, -1):
        if res[i] == 'A' or res[i] == 'B' or res[i] == 'C' or res[i] == 'D':
            # Check if res[i-1] is not a character
            if not (res[i-1] >= 'a' and res[i-1] <= 'z') and not (res[i-1] >= 'A' and res[i-1] <= 'Z'):
                return res[i]
                break
    return None

In [13]:
from collections import Counter

In [20]:
response['answers'] = response['responses'].apply(lambda x: [get_model_choice(i) for i in x])

In [27]:
response[['answers', 'reward_model']].apply(lambda x: len([y for y in x['answers'] if y == x['reward_model']['ground_truth']]), axis=1)

0      4
1      4
2      5
3      3
4      1
      ..
193    1
194    2
195    3
196    2
197    0
Length: 198, dtype: int64

In [1]:
response.iloc[197]['answers'].count('A')

NameError: name 'response' is not defined

In [33]:
print(response.iloc[197]['responses'][2])

<think>
Okay, so I have this problem about two quantum states with energies E1 and E2, and their lifetimes are given as 10^-9 seconds and 10^-8 seconds respectively. The question is asking which energy difference would allow us to clearly distinguish these two levels. The options are A through D, with energies in eV.

First, I need to understand what's being asked. The energy difference between the two states must be such that their lifetimes allow for clear resolution. I remember that the energy of a photon is related to its wavelength by E = hc/λ, but here we're dealing with energy difference and lifetime. 

The key here is probably the concept of the uncertainty principle in quantum mechanics. The uncertainty principle states that the product of the time uncertainty and the energy uncertainty is at least on the order of Planck's constant h. Mathematically, it's Δt * ΔE ≥ h/4π.

Here, the lifetimes are Δt1 and Δt2 for the two states. But since we are looking for the energy difference